# Amazon SageMaker XGBoost Bring Your Own Model
_**Hosting a Pre-Trained XGBoost Model using Amazon SageMaker XGBoost Algorithm Container**_

* Option #1: Deploy to Batch Transform for batch inference
* Option #2: Deploy to Endpoint for real-time prediction

---

---

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Optionally, train a scikit learn XGBoost model](#Optionally,-train-a-scikit-learn-XGBoost-model)
1. [Upload the pre-trained model to S3](#Upload-the-pre-trained-model-to-S3)
1. [Set up hosting for the model](#Set-up-hosting-for-the-model)
1. [Validate the model for use](#Validate-the-model-for-use)




---
## Background

Amazon SageMaker includes functionality to support a hosted notebook environment, distributed, serverless training, and real-time hosting. We think it works best when all three of these services are used together, but they can also be used independently.  Some use cases may only require hosting.  Maybe the model was trained prior to Amazon SageMaker existing, in a different service.

This notebook shows how to use a pre-existing scikit-learn trained XGBoost model with the Amazon SageMaker XGBoost Algorithm container to quickly create a hosted endpoint for that model. Please note that scikit-learn XGBoost model is compatible with SageMaker XGBoost container, whereas other gradient boosted tree models (such as one trained in SparkML) are not.

---
## Setup

Let's start by specifying:

* AWS region.
* The IAM role arn used to give training and hosting access to your data.
* The SageMaker session object used to access resources.
* The S3 bucket that you want to use for training and model data.
* The S3 prefix used to store resources created by the notebook.

In [1]:
import os
import boto3
import re
import json
import sagemaker

from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.transformer import Transformer
from sagemaker.predictor import csv_serializer, json_deserializer

region = boto3.Session().region_name

role = get_execution_role()

session = sagemaker.Session()

bucket = sagemaker.Session().default_bucket()

prefix = 'sagemaker/DEMO-xgboost-byo'

print ("Bucket: " + bucket);

Bucket: sagemaker-us-east-1-572539092864


## Optionally, train a scikit learn XGBoost model

These steps are optional and are needed to generate the scikit-learn model that will eventually be hosted using the SageMaker Algorithm contained. 

### Install XGboost
Note that for conda based installation, you'll need to change the Notebook kernel to the environment with conda and Python3. 

In [2]:
!conda update -y -n base -c defaults conda

Solving environment: done

# All requested packages already installed.



In [3]:
!conda install -y -c conda-forge xgboost==0.90

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::bkcharts==0.2=py36_0
  - defaults/noarch::dask==2.14.0=py_0
  - defaults/linux-64::pytest-arraydiff==0.3=py36h39e3cac_0
  - defaults/linux-64::bottleneck==1.3.2=py36heb32a55_0
  - defaults/linux-64::pywavelets==1.1.1=py36h7b6447c_0
  - defaults/noarch::pytest-astropy==0.8.0=py_0
  - defaults/linux-64::numexpr==2.7.1=py36h423224d_0
  - defaults/linux-64::h5py==2.10.0=py36h7918eee_0
  - defaults/linux-64::pandas==1.0.5=py36h0573a6f_0
  - defaults/linux-64::numpy-base==1.18.1=py36hde5b4d6_1
  - conda-forge/linux-64::xgboost==0.90=py36he1b5a44_4
  - defaults/linux-64::patsy==0.5.1=py36_0
  - defaults/linux-64::scikit-image==0.16.2=py36h0573a6f_0
  - defaults/linux-64::matplotlib-base==3.1.3=py36hef1b27d_0
  - defaults/linux-64::pytables==3.6.1=py36h71ec239_0
  - defaults/linux-64::mkl_fft==1.0.15=py36ha843d7b_0
  - de

### Fetch the dataset

In [ ]:
#import pickle, gzip, numpy, urllib.request, json

# Load the dataset
##urllib.request.urlretrieve("http://deeplearning.net/data/mnist/mnist.pkl.gz", "mnist.pkl.gz")

#### Dataset is already downloaded to local filesystem


In [4]:
import pickle, gzip, numpy, urllib.request, json

f = gzip.open('mnist.pkl.gz', 'rb')
train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
f.close()

### Prepare the dataset for training

In [5]:
%%time

import struct
import io
import boto3

def get_dataset():
  import pickle
  import gzip
  with gzip.open('mnist.pkl.gz', 'rb') as f:
      u = pickle._Unpickler(f)
      u.encoding = 'latin1'
      return u.load()

CPU times: user 8 µs, sys: 2 µs, total: 10 µs
Wall time: 12.2 µs


In [6]:
train_set, valid_set, test_set = get_dataset()

train_X = train_set[0]
train_y = train_set[1]

valid_X = valid_set[0]
valid_y = valid_set[1]

test_X = test_set[0]
test_y = test_set[1]

In [7]:
import numpy as np

# Save training data (numpy array) to local file
np.savetxt('train.csv',
           train_X,
           delimiter=',',
           fmt='%i')

In [8]:
print(f'bucket: {bucket} prefix: {prefix}')

bucket: sagemaker-us-east-1-572539092864 prefix: sagemaker/DEMO-xgboost-byo


In [9]:
# Upload training data file to S3 for batch transformer
train_s3 = session.upload_data('train.csv', 
                            bucket=bucket, 
                            key_prefix='{}/train'.format(prefix))

### Train the XGBClassifier

In [10]:
import xgboost as xgb
import sklearn as sk 

bt = xgb.XGBClassifier(max_depth=5,
                       learning_rate=0.2,
                       n_estimators=10,
                       objective='multi:softmax')   # Setup xgboost model
bt.fit(train_X, train_y, # Train it to our data
       eval_set=[(valid_X, valid_y)], 
       verbose=False)

XGBClassifier(learning_rate=0.2, max_depth=5, n_estimators=10,
              objective='multi:softprob')

### Save the trained model file

Note that the model file name must satisfy the regular expression pattern: `^[a-zA-Z0-9](-*[a-zA-Z0-9])*;`. The model file also need to tar-zipped. 

In [11]:
model_file_name = "DEMO-local-xgboost-model"
bt._Booster.save_model(model_file_name)

In [12]:
!tar czvf model.tar.gz $model_file_name

DEMO-local-xgboost-model


### Upload the pre-trained model to S3

In [13]:
fObj = open("model.tar.gz", 'rb')
key= os.path.join(prefix, model_file_name, 'model.tar.gz')
boto3.Session().resource('s3').Bucket(bucket).Object(key).upload_fileobj(fObj)

In [14]:
print(f'bucket: {bucket} key: {key}')

bucket: sagemaker-us-east-1-572539092864 key: sagemaker/DEMO-xgboost-byo/DEMO-local-xgboost-model/model.tar.gz


## Set up hosting for the model

### Use SageMaker to import and create model
This involves creating a SageMaker model from the model file previously uploaded to S3.

In [15]:
container = get_image_uri(boto3.Session().region_name, 'xgboost', '0.90-2')

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
There is a more up to date SageMaker XGBoost image. To use the newer image, please set 'repo_version'='1.0-1'. For example:
	get_image_uri(region, 'xgboost', '1.0-1').


In [16]:
print(f'Container: {container}')

Container: 683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3


In [17]:
%%time
from time import gmtime, strftime

model_name = model_file_name + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = 'https://s3-{}.amazonaws.com/{}/{}'.format(region,bucket,key)
sm_client = boto3.client('sagemaker')

print (model_url)

primary_container = {
    'Image': container,
    'ModelDataUrl': model_url,
}

create_model_response2 = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response2['ModelArn'])

https://s3-us-east-1.amazonaws.com/sagemaker-us-east-1-572539092864/sagemaker/DEMO-xgboost-byo/DEMO-local-xgboost-model/model.tar.gz
arn:aws:sagemaker:us-east-1:572539092864:model/demo-local-xgboost-model2020-12-16-00-35-08
CPU times: user 38.9 ms, sys: 51 µs, total: 38.9 ms
Wall time: 535 ms


In [18]:
print(f'model_name: {model_name}')

model_name: DEMO-local-xgboost-model2020-12-16-00-35-08


### First, let's deploy the model using Batch Transform 

Instead of deploying to real-time endpoint, we show how to run the model directly via batch transform

In [19]:
# Allocate the Transformer 
xgb_transformer = Transformer(model_name=model_name,
                        instance_count=1,
                        instance_type='ml.m4.xlarge',
                        strategy='MultiRecord',
                        assemble_with='Line',
                        output_path='s3://{}/{}/batch-transform/output'.format(bucket, prefix))

In [20]:
xgb_transformer.transform(train_s3,
                          data_type='S3Prefix',
                          content_type='text/csv',
                          split_type='Line',
                          wait=True)


.......................................................................!


### Optionally, deploy the model to real-time Endpoint

First, we have to create `endpoint-configuration`

SageMaker supports configuring REST endpoints in hosting with multiple models, e.g. for A/B testing purposes. In order to support this, you can create an endpoint configuration, that describes the distribution of traffic across the models, whether split, shadowed, or sampled in some way. In addition, the endpoint configuration describes the instance type required for model deployment.

In [21]:
from time import gmtime, strftime

endpoint_config_name = 'DEMO-XGBoostEndpointConfig-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_config_name)
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m4.xlarge',
        'InitialInstanceCount':1,
        'InitialVariantWeight':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

DEMO-XGBoostEndpointConfig-2020-12-16-00-48-22
Endpoint Config Arn: arn:aws:sagemaker:us-east-1:572539092864:endpoint-config/demo-xgboostendpointconfig-2020-12-16-00-48-22


### Create endpoint
Lastly, you create the endpoint that serves up the model, through specifying the name and configuration defined above. The end result is an endpoint that can be validated and incorporated into production applications. This takes 9-11 minutes to complete.

In [ ]:
%%time
import time

endpoint_name_now = 'DEMO-XGBoostEndpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
endpoint_name = 'DEMO-XGBoostEndpoint-2020-06-15-21-39-19'
print(endpoint_name)

# Comment out creation!
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Status: " + status)

while status=='Creating':
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp['EndpointStatus']
    print("Status: " + status)

print("Arn: " + resp['EndpointArn'])
print("Status: " + status)

## Validate the model for use
Now you can obtain the endpoint from the client library using the result from previous operations and generate classifications from the model using that endpoint.

In [ ]:
runtime_client = boto3.client('runtime.sagemaker')

Lets generate the prediction for a single datapoint. We'll pick one from the test data generated earlier.

In [ ]:
import numpy as np
point_X = test_X[0]
point_X = np.expand_dims(point_X, axis=0)
point_y = test_y[0]
np.savetxt("test_point.csv", point_X, delimiter=",")

In [ ]:
%%time
import json

file_name = 'test_point.csv' #customize to your test file, will be 'mnist.single.test' if use data above

with open(file_name, 'r') as f:
    payload = f.read().strip()

response = runtime_client.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType='text/csv', 
                                   Body=payload)
result = response['Body'].read().decode('ascii')
print('Predicted Class Probabilities: {}.'.format(result))

### Post process the output
Since the result is a string, let's process it to determine the the output class label. 

In [ ]:
floatArr = np.array(json.loads(result))
predictedLabel = np.argmax(floatArr)
print('Predicted Class Label: {}.'.format(predictedLabel))
print('Actual Class Label: {}.'.format(point_y))

### (Optional) Delete the Endpoint

If you're ready to be done with this notebook, please run the delete_endpoint line in the cell below.  This will remove the hosted endpoint you created and avoid any charges from a stray instance being left on.

In [ ]:
# sm_client.delete_endpoint(EndpointName=endpoint_name)